In [2]:

#import math
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# define plt settings
plt.rcParams["font.size"] = 20
plt.rcParams["axes.labelsize"] = 20
plt.rcParams["xtick.labelsize"] = 20
plt.rcParams["ytick.labelsize"] = 20
plt.rcParams["legend.fontsize"] = 20
plt.rcParams["figure.figsize"] = (20,10)

import sklearn 
from sklearn import metrics
from sklearn import datasets
from sklearn import pipeline
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import svm
from sklearn.linear_model import LogisticRegression  # for Logistic Regression algorithm
#from sklearn.cross_validation import train_test_split #to split the dataset for training and testing
#from sklearn.neighbors import KNeighborsClassifier  # for K nearest neighbours
from sklearn.tree import DecisionTreeClassifier #for using Decision Tree Algoithm

import pandas as pd
#import statistics as st

df = pd.read_csv('train.csv')

In [ ]:
df.info()